In [1]:
from functions.sleeper_functions import *
import http.client, urllib.request, urllib.parse
import pandas as pd
from datetime import datetime
from random import sample as sp

In [2]:
scoring_types = ['dynasty_2qb','ppr','2qb']#'ppr','2qb']#,'std','half_ppr']
df = league_details()
df = df[df['scoring_type'].isin(scoring_types) | df['is_superflex'] == True]
df = df[df['scoring_type'].str.startswith('dynasty')]
df['last_message_time'] = pd.to_datetime(df['last_message_time'], unit='ms')

FileNotFoundError: [Errno 2] No such file or directory: 'Files/leagues_info.csv'

In [3]:
df

,league_id,draft_id,teams,scoring_type,is_superflex,last_message_time
70,837120699612114944,837120700627152896,10,dynasty_2qb,True,2022-06-02 17:58:39.867
21750,786701411730776064,786701411730776065,12,dynasty_2qb,True,2022-06-02 17:58:17.234
27517,785499599937908736,785499599937908737,12,dynasty_2qb,True,2022-06-02 17:56:38.028
51198,784357377242734592,834509543349280768,10,dynasty_2qb,False,2022-06-02 17:54:29.170
1,838138453798559744,838142696328646656,12,dynasty_2qb,True,2022-06-02 17:52:35.762
...,...,...,...,...,...,...
54812,777702419760046080,777702421710368768,12,dynasty_2qb,True,2021-12-17 23:49:56.878
55003,775987868891230208,775989377699807232,12,dynasty_2qb,True,2021-12-13 05:21:00.361
55014,775896379028594688,775896380475895808,8,dynasty_2qb,True,2021-12-13 03:03:04.601
55284,772215625597784064,772213478520922112,10,dynasty_2qb,True,2021-12-02 19:24:21.517


In [4]:
#league_id = 720025558011953152
def active_leagues(leagues=None, limit=None, week=None):

    start = datetime.now()
    actives = pd.DataFrame(columns=['league_id','transactions', 'week', 'last_logged_transaction'])
    
    
    ea = pd.read_csv('Files/active_leagues.csv', dtype={'league_id':'str','transactions':'int','week':'int'}, parse_dates=['last_updated'])
    ea = ea[['league_id', 'transactions', 'week', 'type', 'budget', 'day', 'last_updated']]
    sea = len(ea)
    
    
    if limit==None:
        sample=500
    else:
        sample=limit
    
    if leagues==None:
        leagues=ea.league_id.drop_duplicates().to_list()
        lt = 'existing Active list'
        #print('running ' + str(len(leagues)) + ' leagues from existing Active list')
    elif type(leagues)==list:
        leagues=leagues
        lt = 'given list'
        #print('running ' + str(len(leagues)) + ' leagues from given list')
    elif leagues.lower()=='sample':
        l = league_details()
        l = l[l['scoring_type'].isin(['ppr','2qb','std','half_ppr'])]
        l = l.league_id.to_list()
        leagues=sp(l, sample)
        lt = 'sample'
        #print('running ' + str(len(leagues)) + ' leagues from sample')
    
    if limit==None:
        leagues=leagues
    elif limit > len(leagues):
        leagues=leagues
    else:
        leagues=leagues[:limit]
    
    
    print('running ' + str(len(leagues)) + ' leagues from ' + lt)
    
    conn = http.client.HTTPSConnection('api.sleeper.app')
    
    if week==None:
        conn.request("GET", '/v1/state/nfl')
        week = json.loads(conn.getresponse().read()).get('display_week')
    
    idCount = 0

    for league_id in leagues:
        idCount += 1
        if idCount % 100 == 0:
            pct = (idCount/len(leagues))
            span = (datetime.now() - start) / pct
            print(str(idCount) + '/' + str(len(leagues)) + ' (' + "%.2f" % (pct * 100) + '%) ETR: ' + str(start + span))
        
        
        try:
            
            conn.request("GET", '/v1/league/' + str(league_id))
            start_conn = conn.getresponse().read()
            ls = json.loads(start_conn)

            df = pd.DataFrame(ls.get('settings'), index=[league_id])[['waiver_type','waiver_budget','waiver_day_of_week']].reset_index()
            df.columns = ['league_id','type','budget','day']
            
            conn.request("GET", '/v1/league/' + str(league_id) + '/transactions/' + str(week))
            trade_conn = conn.getresponse().read()
            trade = json.loads(trade_conn)

            transactions = pd.DataFrame(trade)
            transactions['time'] = pd.to_datetime(transactions['created'], unit='ms')
            
            last_logged_transaction = max(transactions['time'])
            transactions = len(transactions)
            
            act = pd.DataFrame([[league_id, transactions, week, last_logged_transaction]], columns=['league_id','transactions', 'week', 'last_logged_transaction'])
            act = act.merge(df, on='league_id',how='left')
            act['last_updated'] = datetime.now()

            actives = actives.append(act)

        except:
            pass
    
    
    #print(actives.head())
    ea = ea.append(actives)
    ea = ea.sort_values(by='last_updated', ascending=False)
    ea = ea.drop_duplicates(subset=['league_id']).reset_index(drop=True)
    
    leagues = league_details().dropna(subset=['league_id'])
    #leagues['league_id'] = leagues['league_id'].astype('int')
    ea = ea.merge(leagues[['league_id', 'scoring_type','is_superflex']], on='league_id', how='left')
    ea = ea.dropna(subset=['scoring_type'])
    
    eea = len(ea)
    
    print('Started: ' + str(sea) + ' | Added: ' + str(eea - sea) + ' | Final: ' + str(eea))
    
    ea.to_csv('Files/active_leagues.csv', index=False)
    
    #league_details()
    
    conn.close()
    print(datetime.now()-start)
    
    return ea

In [52]:
#als = pd.read_csv('Files/active_leagues.csv')
#als

In [53]:
#import numpy as np
#league_ids = als[als['last_logged_transaction'].isnull()].league_id.drop_duplicates().to_list()
#len(league_ids)

In [5]:
league_ids = df.league_id.drop_duplicates().to_list()

active_leagues(leagues = league_ids)

running 4342 leagues from given list
100/4342 (2.30%) ETR: 2022-06-28 11:35:49.854883
200/4342 (4.61%) ETR: 2022-06-28 11:35:17.455574
300/4342 (6.91%) ETR: 2022-06-28 11:35:00.462375
400/4342 (9.21%) ETR: 2022-06-28 11:34:42.680083
500/4342 (11.52%) ETR: 2022-06-28 11:34:33.285476
600/4342 (13.82%) ETR: 2022-06-28 11:35:00.346495
700/4342 (16.12%) ETR: 2022-06-28 11:34:59.379263
800/4342 (18.42%) ETR: 2022-06-28 11:34:54.089063
900/4342 (20.73%) ETR: 2022-06-28 11:34:57.038617
1000/4342 (23.03%) ETR: 2022-06-28 11:35:04.416557
1100/4342 (25.33%) ETR: 2022-06-28 11:35:11.996105
1200/4342 (27.64%) ETR: 2022-06-28 11:35:15.230744
1300/4342 (29.94%) ETR: 2022-06-28 11:35:20.781199
1400/4342 (32.24%) ETR: 2022-06-28 11:35:18.730959
1500/4342 (34.55%) ETR: 2022-06-28 11:35:22.964799
1600/4342 (36.85%) ETR: 2022-06-28 11:35:29.042128
1700/4342 (39.15%) ETR: 2022-06-28 11:35:30.689009
1800/4342 (41.46%) ETR: 2022-06-28 11:35:26.914341
1900/4342 (43.76%) ETR: 2022-06-28 11:35:34.570472
2000/43

,league_id,transactions,week,type,budget,day,last_updated,last_logged_transaction,scoring_type,is_superflex
0,775896379028594688,11,1,2.0,100.0,2.0,2022-06-28 11:35:20.638792,2022-05-03 17:18:46.085,dynasty_2qb,True
1,776966210507120640,27,1,1.0,100.0,2.0,2022-06-28 11:35:20.259329,2022-05-20 15:13:10.046,dynasty_2qb,True
2,774692272020267008,8,1,0.0,100.0,2.0,2022-06-28 11:35:19.990605,2021-12-16 23:23:04.077,dynasty_2qb,False
3,778544698561306624,38,1,2.0,100.0,3.0,2022-06-28 11:35:19.700318,2022-01-07 01:40:31.046,dynasty_2qb,True
4,784646273977409536,6,1,0.0,100.0,2.0,2022-06-28 11:35:19.517234,2022-01-06 17:28:04.231,dynasty_2qb,True
...,...,...,...,...,...,...,...,...,...,...
4811,784118539786248192,0,6,1.0,100.0,0.0,2022-03-15 10:03:10.666989,NaT,ppr,False
4812,789614048831537152,0,6,0.0,100.0,2.0,2022-03-15 10:03:09.981659,NaT,2qb,True
4813,776259929601388544,0,6,0.0,100.0,2.0,2022-03-15 10:03:07.996517,NaT,ppr,False
4814,797561877243752448,0,6,0.0,100.0,2.0,2022-03-15 10:03:03.080148,NaT,ppr,False


In [47]:
lim = 100
tries = 2


for attempt in range(tries):
    used = pd.read_csv('Files/active_leagues.csv')
    used = used[used['week'] < 10].league_id.to_list()
    print('-------- Loop ' + str(attempt + 1) + ' --------')
    for scoring in scoring_types:
        l = df[df['scoring_type'] == scoring]
        print(scoring)
        leagues = l.league_id.to_list()
        al = active_leagues(leagues=leagues, limit=lim, week=6)

-------- Loop 1 --------
dynasty_2qb
running 100 leagues from given list
100/100 (100.00%) ETR: 2022-06-06 16:43:43.427125
Started: 4816 | Added: 0 | Final: 4816
0:00:20.460118
-------- Loop 2 --------
dynasty_2qb
running 100 leagues from given list
100/100 (100.00%) ETR: 2022-06-06 16:43:58.255726
Started: 4816 | Added: 0 | Final: 4816
0:00:14.768480


In [48]:
al = al.drop_duplicates()
#
al
#al[(al['week']==9) & (al['scoring_type'].isin(['ppr','2qb']))].dropna(subset=['type'])

,league_id,transactions,week,type,budget,day,last_updated,last_logged_transaction,scoring_type,is_superflex
0,775896379028594688,11,1,2.0,100.0,2.0,2022-06-06 13:13:47.607920,NaN,dynasty_2qb,True
1,776966210507120640,27,1,1.0,100.0,2.0,2022-06-06 13:13:47.174752,NaN,dynasty_2qb,True
2,774692272020267008,8,1,0.0,100.0,2.0,2022-06-06 13:13:46.927419,NaN,dynasty_2qb,False
3,778544698561306624,38,1,2.0,100.0,3.0,2022-06-06 13:13:46.603353,NaN,dynasty_2qb,True
4,784646273977409536,6,1,0.0,100.0,2.0,2022-06-06 13:13:46.423480,NaN,dynasty_2qb,True
...,...,...,...,...,...,...,...,...,...,...
4811,784118539786248192,0,6,1.0,100.0,0.0,2022-03-15 10:03:10.666989,NaN,ppr,False
4812,789614048831537152,0,6,0.0,100.0,2.0,2022-03-15 10:03:09.981659,NaN,2qb,True
4813,776259929601388544,0,6,0.0,100.0,2.0,2022-03-15 10:03:07.996517,NaN,ppr,False
4814,797561877243752448,0,6,0.0,100.0,2.0,2022-03-15 10:03:03.080148,NaN,ppr,False


In [14]:
al.columns

Index(['league_id', 'transactions', 'week', 'type', 'budget', 'day',
       'last_updated', 'scoring_type', 'is_superflex'],
      dtype='object')

In [16]:
al.groupby('scoring_type')['league_id'].count()

scoring_type
2qb             129
dynasty_2qb    4293
dynasty_ppr      10
half_ppr         14
ppr             172
std              34
Name: league_id, dtype: int64

In [8]:
test = al.sort_values(by=['week','transactions'], ascending=[False,False]).drop_duplicates(subset='league_id')

In [9]:
test['last_updated'] = datetime.now()
test

,league_id,transactions,week,type,budget,day,last_updated,scoring_type,is_superflex
11335,685808640371818496,190,6,0.0,100.0,0.0,2021-11-23 17:00:02.310707,2qb,True
13767,649941619793031168,149,6,0.0,100.0,0.0,2021-11-23 17:00:02.310707,ppr,False
13475,700274850681389056,131,6,2.0,100.0,2.0,2021-11-23 17:00:02.310707,2qb,True
16712,722971814690652160,130,6,0.0,100.0,1.0,2021-11-23 17:00:02.310707,ppr,False
6637,731355591838142464,121,6,2.0,1000.0,0.0,2021-11-23 17:00:02.310707,2qb,True
...,...,...,...,...,...,...,...,...,...
36862,735012342835843072,0,1,NaN,NaN,NaN,2021-11-23 17:00:02.310707,ppr,False
36863,727835169003769856,0,1,NaN,NaN,NaN,2021-11-23 17:00:02.310707,ppr,False
36864,725914254938591232,0,1,NaN,NaN,NaN,2021-11-23 17:00:02.310707,ppr,False
36865,734694769594396672,0,1,NaN,NaN,NaN,2021-11-23 17:00:02.310707,half_ppr,False


In [79]:
test.to_csv('Files/active_leagues.csv', index=False)

In [80]:
act

,league_id,transactions,week,type,budget,day,last_updated,scoring_type,is_superflex
0,723491469624700928,0,4,0.0,100.0,0.0,2021-09-28 16:38:32.971696,2qb,True
1,738171284516737024,0,4,0.0,100.0,2.0,2021-09-28 16:38:32.626695,ppr,False
2,690237954869760000,0,4,0.0,100.0,2.0,2021-09-28 16:38:32.261515,ppr,False
3,650090411397824512,0,4,1.0,100.0,2.0,2021-09-28 16:38:32.002777,ppr,False
4,725057088883068928,0,4,2.0,100.0,2.0,2021-09-28 16:38:31.635935,ppr,False
...,...,...,...,...,...,...,...,...,...
35868,712776074009874432,0,1,NaN,NaN,NaN,NaT,std,False
35869,733818794232942592,0,1,NaN,NaN,NaN,NaT,ppr,False
35870,720744742207426560,0,1,NaN,NaN,NaN,NaT,ppr,False
35871,643904009497083904,0,1,NaN,NaN,NaN,NaT,ppr,False
